In [ ]:
import listing_envs
from typing import Dict, List, Union
import os
import json

Find experiments.

In [ ]:
# Get list of experiment environments.
LIST_OF_EXPERIMENT_ENVIRONMENTS: List[str] = listing_envs.get_list_of_constraints_selection_env_paths()
print(
    "There are",
    "`" + str(len(LIST_OF_EXPERIMENT_ENVIRONMENTS)) + "`",
    "created experiment environments in `../experiments`",
)
LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS: List[str] = [
    env
    for env in LIST_OF_EXPERIMENT_ENVIRONMENTS
    if not os.path.exists(env+".done")
]
print(
    "There are",
    "`" + str(len(LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS)) + "`",
    "todo experiment environments in `../experiments`",
)
LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS

List errors

In [ ]:
errors = {
    "dict_of_clustering_performances.json": [],
    "dict_of_clustering_results.json": [],
    "dict_of_constraints_effective.json": [],
    "dict_of_errors.json": [],
    "dict_of_samplings.json": [],
}

for file in errors:
    for env in LIST_OF_EXPERIMENT_ENVIRONMENTS:
        try:
            with open(env + file, "r") as f:
                json.load(f)
        except:
            errors[file].append(env)
errors

Fix `dict_of_clustering_results.json`.

> (_manuel_)

Fix `dict_of_constraints_effective.json`.

> (_manuel_)

Fix `dict_of_errors.json`.

In [ ]:
for env in errors["dict_of_errors.json"]:
    
    with open(env + "dict_of_constraints_effective.json", "r") as fr:
        dict_of_constraints_effective = json.load(fr)
    new_errors = {
        iteration: [
            [constraint[0], constraint[1]]
            for constraint in constraints_effective
            if (
                (not constraint[4] and constraint[3])  # not conflict ==> no fix ==> is error
                or (constraint[4] and not constraint[3])  # conflict ==> fix ==> is not conflict
            )
        ]
        for iteration, constraints_effective in dict_of_constraints_effective.items()
    }
    with open(env + "dict_of_errors.json", "w") as fw:
        json.dump(new_errors, fw)

Fix `dict_of_samplings.json`.

In [ ]:
for env in errors["dict_of_samplings.json"]:
    
    with open(env + "dict_of_constraints_effective.json", "r") as fr:
        dict_of_constraints_effective = json.load(fr)
    new_sampling = {
        iteration: [
            [constraint[0], constraint[1]]
            for constraint in constraints_effective
        ]
        for iteration, constraints_effective in dict_of_constraints_effective.items()
    }
    with open(env + "dict_of_samplings.json", "w") as fw:
        json.dump(new_sampling, fw)

Min-Max iterations.

In [ ]:
range_iterations = {}
for env in LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS:
    range_iterations[env] = []
    for file in errors: 
        try:
            with open(env + file, "r") as f:
                i = sorted(json.load(f).keys())[-1]
                range_iterations[env].append(i)
        except:
            print(env+file)
minmax_iterations = {
    env: [
        min(range_iterations[env]),
        max(range_iterations[env])
    ]
    for env in LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS
}

In [ ]:
for env in LIST_OF_TODO_EXPERIMENT_ENVIRONMENTS:
    with open(env+"dict_of_clustering_performances.json", "r") as fr:
        dict_of_clustering_performances = json.load(fr)
    
    new_dict_of_clustering_performances = {
        k:v
        for k,v in dict_of_clustering_performances.items()
        if int(k) <= int(minmax_iterations[env][0])
    }
    with open(env+"dict_of_clustering_performances.json", "w") as fw:
        json.dump(new_dict_of_clustering_performances, fw)